In [1]:
import pandas as pd

In [3]:
pip install gspread oauth2client

Note: you may need to restart the kernel to use updated packages.


In [4]:
from google.oauth2.service_account import Credentials

In [5]:
# Define the scope and authenticate using the credentials JSON file
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file(r'C:\Users\iccen\Desktop\DA-Task\da-assessment-437914-51755eea63c3.json', scopes=scope)

In [6]:
import gspread

In [11]:
# Authorize the client
client = gspread.authorize(creds)

In [13]:
pip install gspread google-auth

Note: you may need to restart the kernel to use updated packages.


In [21]:
spreadsheet = client.open("data cleanup assignment")

In [23]:
# Load data from different google sheets into pandas DataFrames
sheet_DE = spreadsheet.worksheet("DE")
sheet_AT= spreadsheet.worksheet("AT")
sheet_CH = spreadsheet.worksheet("CH")
sheet_mixed = spreadsheet.worksheet("mixed")

In [27]:
# Convert the sheet data into pandas DataFrames
df1 = pd.DataFrame(sheet_DE.get_all_records())  
df2 = pd.DataFrame(sheet_AT.get_all_records())  
df3 = pd.DataFrame(sheet_CH.get_all_records())   
df4 = pd.DataFrame(sheet_mixed.get_all_records())

In [29]:
#check the loaded data
print(df1.head())

                       firma                 street    plz       city  \
0    Abschleppdienst Arnolds  Völlesbruchstrasse 19  52152  Simmerath   
1              AAS-Fink GmbH            Morsbach 39  42857  Remscheid   
2  Allfolia Deutschland GmbH            Morsbach 39  42857  Remscheid   
3    Autohaus Hentschel GmbH  Vahrenwalder Str. 141  30165   Hannover   
4      Autohaus Schmohl GmbH     Potsdamer Str. 175  14469    Potsdam   

  old_ctry           telefon  
0       DE   0049177-8754883  
1       DE        172.208606  
2       DE      491734636476  
3       DE   +491786221169  
4       DE  0049/160466 6050  


In [31]:
# Standardize the column names before merging

# For df1 (sheet_DE)
df1.rename(columns={
    'old_ctry': 'country'
}, inplace=True)

# For df4 (sheet_mixed)
df4.rename(columns={
    'landesvorwahl': 'country',
    'telefonnr': 'telefon'
}, inplace=True)

In [33]:
#check the renamed columns for df4
print(df4.columns.tolist())

['firma', 'street', 'plz', 'city', 'country', 'telefon', 'anrede', 'vorname', 'nachname']


In [35]:
# Check the unique values in country column
print("Unique values in df1 country column:", df1['country'].unique())
print("Unique values in df2 country column:", df2['country'].unique())
print("Unique values in df3 country column:", df3['country'].unique())
print("Unique values in df4 country column:", df4['country'].unique())

Unique values in df1 country column: ['DE']
Unique values in df2 country column: ['AT']
Unique values in df3 country column: ['CH']
Unique values in df4 country column: [49 43 41 12 '' 39 44]


In [37]:
# check the count of the different values in df4
print(df4['country'].value_counts())

country
49    3429
41     114
43      70
         2
12       1
39       1
44       1
Name: count, dtype: int64


In [39]:
# Remove rows with 39, 44, 12, 4, assuming our market focus is in DE, AT and CH country codes.
df4 = df4[~df4['country'].isin(['39', '12', '44', '4'])]

In [41]:
# check the count of the different values in df4
print(df4['country'].value_counts())

country
49    3429
41     114
43      70
         2
12       1
39       1
44       1
Name: count, dtype: int64


In [47]:
# Remove all other countries assuming we are focused on DE, AT and CH
df4 = df4[~df4['country'].isin([39, 12, 44, 4, ""])]

In [49]:
# check the count of the different values in df4
print(df4['country'].value_counts())

country
49    3429
41     114
43      70
Name: count, dtype: int64


In [51]:
# Concatenate the DataFrames df1 to df4 together and reset the index
df_combined = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [53]:
print(df_combined.tail())

     firma                  street    plz      city country       telefon  \
4765                                44809                49   23490401110   
4766                                71336                49  715116799710   
4767        Dr.- Kessel-Straße 11   40878  Ratingen      49   21029434997   
4768             Bismarckstr. 104   47443     Moers      49     284151254   
4769                 Herrnacker 7   61276   Weilrod      49   60833299884   

     anrede vorname nachname  
4765                          
4766                          
4767                          
4768                          
4769                          


In [55]:
#import regex to handle the cleaning of the data
import re

In [57]:
# Change the telefon column to string before cleaning
df_combined['telefon'] = df_combined['telefon'].astype(str)

In [59]:
# Remove all non numeric number and non plus
df_combined["telefon2"] = df_combined["telefon"].str.replace(r'[^0-9+]', '', regex=True)

In [61]:
print(df_combined[['telefon', 'telefon2']])

               telefon        telefon2
0      0049177-8754883  00491778754883
1          172.2086056      1722086056
2         491734636476    491734636476
3      +491786221169   +491786221169
4     0049/160466 6050  00491604666050
...                ...             ...
4765       23490401110     23490401110
4766      715116799710    715116799710
4767       21029434997     21029434997
4768         284151254       284151254
4769       60833299884     60833299884

[4770 rows x 2 columns]


In [63]:
df_combined['country'].unique()

array(['DE', 'AT', 'CH', 49, 43, 41], dtype=object)

In [65]:
# Replace 49, 43 and 41 with DE, AT and CH respectively
df_combined['country'] = df_combined['country'].astype(str).replace({'49': 'DE', '43': 'AT', '41': 'CH'})

In [67]:
df_combined['country'].unique()

array(['DE', 'AT', 'CH'], dtype=object)

In [69]:
# Check the count of duplicate records
duplicate_count = df_combined.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 26


In [71]:
# Remove duplicates while keeping the last entry
df_combined = df_combined.drop_duplicates(keep='last')

In [73]:
# Replace '+' with '00' in the telefon2 column
df_combined['telefon2'] = df_combined['telefon2'].str.replace('+', '00', regex=False)

# Display the updated DataFrame
print(df_combined[['country', 'telefon2']])

     country        telefon2
0         DE  00491778754883
1         DE      1722086056
2         DE    491734636476
3         DE  00491786221169
4         DE  00491604666050
...      ...             ...
4765      DE     23490401110
4766      DE    715116799710
4767      DE     21029434997
4768      DE       284151254
4769      DE     60833299884

[4744 rows x 2 columns]


In [89]:
pd.set_option('display.max_rows', None)
print(df_combined[['country', 'telefon2']].head(100))

   country        telefon2
0       DE  00491778754883
1       DE      1722086056
2       DE    491734636476
3       DE  00491786221169
4       DE  00491604666050
5       DE  00491751165373
6       DE    491735330785
7       DE    491750397296
8       DE  00491799703167
9       DE  00491748987808
10      DE  00491769142078
11      DE    491600795434
12      DE  17800996780049
13      DE  00491789135039
14      DE  00491770634678
15      DE  00491733091370
16      DE    491739990273
17      DE    491779716592
18      DE      1790018238
19      DE  17606998740049
20      DE  00491774376638
21      DE    004578976173
22      DE    491521029742
23      DE  15222170510049
24      DE  17949358360049
25      DE      1770859851
26      DE    491776659132
27      DE  17477777300049
28      DE      1526516945
29      DE      1769029428
30      DE    491792236285
31      DE    491609647354
32      DE    491520780234
33      DE  00491607443158
34      DE  00491723584620
35      DE    005820795177
3

In [75]:
# Move '0049' from the end to the start for country 'DE'
df_combined.loc[df_combined['country'] == 'DE', 'telefon2'] = df_combined.loc[
    df_combined['country'] == 'DE', 'telefon2'].apply(
        lambda x: '0049' + x[:-4] if x.endswith('0049') else x
    )

In [77]:
# Format telefon2 to telefon3 and make all the DE numbers start with 0049
df_combined.loc[df_combined['country'] == 'DE', 'telefon3'] = df_combined.loc[
    df_combined['country'] == 'DE', 'telefon2'].apply(
        lambda x: x if x.startswith('0049') else
        ('00' + x[2:] if x.startswith('00') and '49' not in x[2:] else 
        ('0049' + x if not x.startswith(('00', '49')) else 
        ('00' + x if x.startswith('49') else x))))

In [79]:
print(df_combined[['country', 'telefon2', 'telefon3']].head(10))

  country        telefon2        telefon3
0      DE  00491778754883  00491778754883
1      DE      1722086056  00491722086056
2      DE    491734636476  00491734636476
3      DE  00491786221169  00491786221169
4      DE  00491604666050  00491604666050
5      DE  00491751165373  00491751165373
6      DE    491735330785  00491735330785
7      DE    491750397296  00491750397296
8      DE  00491799703167  00491799703167
9      DE  00491748987808  00491748987808


In [81]:
# Format telefon3 further for country 'DE'
df_combined.loc[df_combined['country'] == 'DE', 'telefon4'] = df_combined.loc[
    df_combined['country'] == 'DE', 'telefon3'].apply(
        lambda x: x if x.startswith('0049') else
        (x[:2] + '49' + x[2:] if x.startswith('00') and not x.startswith('0049') else 
        ('00' + x if x.startswith('49') else 
        ('0049' + x if not x.startswith(('00', '49')) else x)))
    )

In [83]:
print(df_combined[['country', 'telefon2', 'telefon3', 'telefon4']].head(100))

   country        telefon2        telefon3        telefon4
0       DE  00491778754883  00491778754883  00491778754883
1       DE      1722086056  00491722086056  00491722086056
2       DE    491734636476  00491734636476  00491734636476
3       DE  00491786221169  00491786221169  00491786221169
4       DE  00491604666050  00491604666050  00491604666050
..     ...             ...             ...             ...
95      DE  00491751349455  00491751349455  00491751349455
96      DE    491604577921  00491604577921  00491604577921
97      DE    003997553152    003997553152  00493997553152
98      DE  00491521656147  00491521656147  00491521656147
99      DE  00491523969801  00491523969801  00491523969801

[100 rows x 4 columns]


In [85]:
# Move '0043' from the end to the start for country 'AT'
df_combined.loc[df_combined['country'] == 'AT', 'telefon2'] = df_combined.loc[
    df_combined['country'] == 'AT', 'telefon2'].apply(
        lambda x: '0043' + x[:-4] if x.endswith('0043') else x
    )

In [87]:
# Format telefon2 based on the specified conditions for country 'AT'
df_combined.loc[df_combined['country'] == 'AT', 'telefon4'] = df_combined.loc[
    df_combined['country'] == 'AT', 'telefon2'].apply(
        lambda x: x if x.startswith('0043') else  #If it starts with 0043, leave it unchanged.
        ('0043' + x[2:] if x.startswith('00') else  #If it starts with 00 (and is not already 0043), remove 00 and add 0043
        ('0043' + x[2:] if x.startswith('43') else  #If it starts with 43, remove  the 43 and add 0043
        ('0043' + x if not x.startswith(('00', '43')) else x)))  #If it doesn’t start with 00 or 43, prepend 0043
)

In [89]:
print(df_combined[df_combined['country'] == 'AT'][['country', 'telefon2', 'telefon4']].head(100))

    country        telefon2        telefon4
865      AT  00436640020108  00436640020108
866      AT      6643019220  00436643019220
867      AT  00436764429785  00436764429785
868      AT    436998521286  00436998521286
869      AT    436763066695  00436763066695
..      ...             ...             ...
960      AT    436500567279  00436500567279
961      AT    436641367242  00436641367242
962      AT    436995515166  00436995515166
963      AT    436505938972  00436505938972
964      AT    004208712676  00434208712676

[100 rows x 3 columns]


In [91]:
print(df_combined[df_combined['country'] == 'CH'][['country', 'telefon2']].head(50))

     country       telefon2
1065      CH      300337878
1066      CH    00415837154
1067      CH    00037950679
1068      CH  0041788579115
1069      CH    41760352498
1070      CH  0041792487154
1071      CH    41760529174
1072      CH    41785232139
1073      CH       76242521
1074      CH    00411583869
1075      CH               
1076      CH  0041796885332
1077      CH  7805598100041
1078      CH    41746737366
1079      CH  0041783283043
1080      CH      783585206
1081      CH      782581264
1082      CH       08565856
1083      CH  7429698360041
1084      CH      418361971
1085      CH    00581054276
1086      CH    00415873606
1087      CH  0041761451509
1088      CH    00418206960
1089      CH    41762836124
1090      CH      796332724
1091      CH    41760985870
1092      CH      419218554
1093      CH        8883738
1094      CH    00700566081
1095      CH  0041813437327
1096      CH    00415220366
1097      CH    41769966950
1098      CH  0041815564738
1099      CH  004181

In [93]:
# Repeat same formatting for CH

In [95]:
# Format telefon4 based on the specified conditions for country 'CH'
df_combined.loc[df_combined['country'] == 'CH', 'telefon4'] = df_combined.loc[
    df_combined['country'] == 'CH', 'telefon2'].apply(
        lambda x: x if x.startswith('0041') else
        (x[2:] if x.startswith('00') and not x.startswith('0041') else
        ('0041' + x[2:] if x.startswith('41') else 
        ('0041' + x if not x.startswith(('00', '41')) else x)))
    )

In [97]:
print(df_combined[df_combined['country'].isin(['CH'])][['country', 'telefon4']])

     country         telefon4
1065      CH    0041300337878
1066      CH      00415837154
1067      CH        037950679
1068      CH    0041788579115
1069      CH    0041760352498
...      ...              ...
4284      CH  004128229776942
4285      CH    0041275271212
4286      CH    0041786176981
4287      CH    0041562210015
4288      CH    0041794727713

[202 rows x 2 columns]


In [99]:
# Remove '0041' from the end of telefon4 for country 'CH'
df_combined.loc[df_combined['country'] == 'CH', 'telefon4'] = df_combined.loc[
    df_combined['country'] == 'CH', 'telefon4'].apply(
        lambda x: x[:-4] if x.endswith('0041') else x
    )

In [101]:
print(df_combined[df_combined['country'].isin(['CH'])][['country', 'telefon4']])

     country         telefon4
1065      CH    0041300337878
1066      CH      00415837154
1067      CH        037950679
1068      CH    0041788579115
1069      CH    0041760352498
...      ...              ...
4284      CH  004128229776942
4285      CH    0041275271212
4286      CH    0041786176981
4287      CH    0041562210015
4288      CH    0041794727713

[202 rows x 2 columns]


In [103]:
# Set telefon4 to blank if it contains only '0049', '0043', or '0041'
df_combined.loc[df_combined['telefon4'].isin(['0049', '0043', '0041']), 'telefon4'] = ''

In [105]:
print(df_combined[df_combined['country'].isin(['DE', 'AT','CH'])][['country', 'telefon2', 'telefon4']])

     country        telefon2          telefon4
0         DE  00491778754883    00491778754883
1         DE      1722086056    00491722086056
2         DE    491734636476    00491734636476
3         DE  00491786221169    00491786221169
4         DE  00491604666050    00491604666050
...      ...             ...               ...
4765      DE     23490401110   004923490401110
4766      DE    715116799710  0049715116799710
4767      DE     21029434997   004921029434997
4768      DE       284151254     0049284151254
4769      DE     60833299884   004960833299884

[4744 rows x 3 columns]


In [107]:
# Display all column names
print(df_combined.columns.tolist())

['firma', 'street', 'plz', 'city', 'country', 'telefon', 'anrede', 'vorname', 'nachname', 'telefon2', 'telefon3', 'telefon4']


In [109]:
df_combined = df_combined.drop(columns=['telefon3'])

In [111]:
print(df_combined.head())

                       firma                 street    plz       city country  \
0    Abschleppdienst Arnolds  Völlesbruchstrasse 19  52152  Simmerath      DE   
1              AAS-Fink GmbH            Morsbach 39  42857  Remscheid      DE   
2  Allfolia Deutschland GmbH            Morsbach 39  42857  Remscheid      DE   
3    Autohaus Hentschel GmbH  Vahrenwalder Str. 141  30165   Hannover      DE   
4      Autohaus Schmohl GmbH     Potsdamer Str. 175  14469    Potsdam      DE   

            telefon anrede vorname nachname        telefon2        telefon4  
0   0049177-8754883    NaN     NaN      NaN  00491778754883  00491778754883  
1       172.2086056    NaN     NaN      NaN      1722086056  00491722086056  
2      491734636476    NaN     NaN      NaN    491734636476  00491734636476  
3   +491786221169    NaN     NaN      NaN  00491786221169  00491786221169  
4  0049/160466 6050    NaN     NaN      NaN  00491604666050  00491604666050  


In [113]:
# Replace NaN values with blank strings for specified columns
df_combined[['anrede', 'vorname', 'nachname']] = df_combined[['anrede', 'vorname', 'nachname']].fillna('')

In [115]:
# Create a new column 'country_code' with the first 4 digits of 'telefon4'
df_combined['country_code'] = df_combined['telefon4'].str[:4]

# Remove the first 4 digits from 'telefon4' to get the local number
df_combined['telefon4'] = df_combined['telefon4'].str[4:]

In [117]:
print(df_combined.head())

                       firma                 street    plz       city country  \
0    Abschleppdienst Arnolds  Völlesbruchstrasse 19  52152  Simmerath      DE   
1              AAS-Fink GmbH            Morsbach 39  42857  Remscheid      DE   
2  Allfolia Deutschland GmbH            Morsbach 39  42857  Remscheid      DE   
3    Autohaus Hentschel GmbH  Vahrenwalder Str. 141  30165   Hannover      DE   
4      Autohaus Schmohl GmbH     Potsdamer Str. 175  14469    Potsdam      DE   

            telefon anrede vorname nachname        telefon2    telefon4  \
0   0049177-8754883                          00491778754883  1778754883   
1       172.2086056                              1722086056  1722086056   
2      491734636476                            491734636476  1734636476   
3   +491786221169                          00491786221169  1786221169   
4  0049/160466 6050                          00491604666050  1604666050   

  country_code  
0         0049  
1         0049  
2         0

In [119]:
# Rename the column 'telefone4' to 'local_phone_number'
df_combined.rename(columns={'telefon4': 'local_phone_number'}, inplace=True)

In [121]:
print(df_combined.head())

                       firma                 street    plz       city country  \
0    Abschleppdienst Arnolds  Völlesbruchstrasse 19  52152  Simmerath      DE   
1              AAS-Fink GmbH            Morsbach 39  42857  Remscheid      DE   
2  Allfolia Deutschland GmbH            Morsbach 39  42857  Remscheid      DE   
3    Autohaus Hentschel GmbH  Vahrenwalder Str. 141  30165   Hannover      DE   
4      Autohaus Schmohl GmbH     Potsdamer Str. 175  14469    Potsdam      DE   

            telefon anrede vorname nachname        telefon2  \
0   0049177-8754883                          00491778754883   
1       172.2086056                              1722086056   
2      491734636476                            491734636476   
3   +491786221169                          00491786221169   
4  0049/160466 6050                          00491604666050   

  local_phone_number country_code  
0         1778754883         0049  
1         1722086056         0049  
2         1734636476      

In [123]:
# Rearrange the columns in the specified order
df_combined = df_combined[['firma', 'street', 'plz', 'city', 'country', 'country_code', 'local_phone_number', 'anrede', 'vorname', 'nachname']]

In [125]:
# Print the column names to confirm the order
print(df_combined.columns.tolist())

['firma', 'street', 'plz', 'city', 'country', 'country_code', 'local_phone_number', 'anrede', 'vorname', 'nachname']


In [127]:
#Change df name to lead_data_cleaned
df_lead_data_cleaned = df_combined

In [129]:
#Save cleaned data to .csv output
df_lead_data_cleaned.to_csv('df_lead_data_cleaned .csv', index=False)

In [133]:
# For country 'CH', if country_code is not '0041', prepend it to local_phone_number
df_lead_data_cleaned.loc[
    (df_lead_data_cleaned['country'] == 'CH') & (df_lead_data_cleaned['country_code'] != '0041'),
    'local_phone_number'
] = df_lead_data_cleaned.loc[
    (df_lead_data_cleaned['country'] == 'CH') & (df_lead_data_cleaned['country_code'] != '0041'),
    'country_code'
].astype(str) + df_lead_data_cleaned.loc[
    (df_lead_data_cleaned['country'] == 'CH') & (df_lead_data_cleaned['country_code'] != '0041'),
    'local_phone_number'
].astype(str)

In [135]:
# For country 'CH', replace country_code with '0041' if it is not '0041'
df_lead_data_cleaned.loc[
    (df_lead_data_cleaned['country'] == 'CH') & (df_lead_data_cleaned['country_code'] != '0041'), 
    'country_code'
] = '0041'

In [143]:
#Save cleaned data to .csv output
df_lead_data_cleaned.to_csv('df_lead_data_cleaned.csv', index=False)

In [145]:
# Ensure country_code is a string with leading zeros
df_lead_data_cleaned['country_code'] = df_lead_data_cleaned['country_code'].replace('', '0000')

# Ensure country_code is a string with leading zeros
df_lead_data_cleaned['country_code'] = df_lead_data_cleaned['country_code'].apply(lambda x: f"{int(x):04d}" if x.isdigit() else x)

In [147]:
#Save cleaned data to .csv output
df_lead_data_cleaned.to_csv('df_lead_data_cleaned.csv', index=False)

In [149]:
print(df_lead_data_cleaned.head())

                       firma                 street    plz       city country  \
0    Abschleppdienst Arnolds  Völlesbruchstrasse 19  52152  Simmerath      DE   
1              AAS-Fink GmbH            Morsbach 39  42857  Remscheid      DE   
2  Allfolia Deutschland GmbH            Morsbach 39  42857  Remscheid      DE   
3    Autohaus Hentschel GmbH  Vahrenwalder Str. 141  30165   Hannover      DE   
4      Autohaus Schmohl GmbH     Potsdamer Str. 175  14469    Potsdam      DE   

  country_code local_phone_number anrede vorname nachname  
0         0049         1778754883                          
1         0049         1722086056                          
2         0049         1734636476                          
3         0049         1786221169                          
4         0049         1604666050                          


In [155]:
# If any DE column has only 49, change it to 0049
df_lead_data_cleaned['country_code'] = df_lead_data_cleaned.apply(
    lambda row: '0049' if row['country'] == 'DE' and row['country_code'] == 49 else row['country_code'],
    axis=1
)

In [157]:
print(df_lead_data_cleaned[df_lead_data_cleaned['country'] == 'DE'][['country', 'country_code', 'local_phone_number']].head())

  country country_code local_phone_number
0      DE         0049         1778754883
1      DE         0049         1722086056
2      DE         0049         1734636476
3      DE         0049         1786221169
4      DE         0049         1604666050


In [159]:
print(df_lead_data_cleaned[df_lead_data_cleaned['country'] == 'AT'][['country', 'country_code', 'local_phone_number']].head())

    country country_code local_phone_number
865      AT         0043         6640020108
866      AT         0043         6643019220
867      AT         0043         6764429785
868      AT         0043         6998521286
869      AT         0043         6763066695


In [161]:
print(df_lead_data_cleaned[df_lead_data_cleaned['country'] == 'CH'][['country', 'country_code', 'local_phone_number']].head())

     country country_code local_phone_number
1065      CH         0041          300337878
1066      CH         0041            5837154
1067      CH         0041          037950679
1068      CH         0041          788579115
1069      CH         0041          760352498


In [165]:
#Save cleaned data to .csv output
df_lead_data_cleaned.to_csv('lead_data_cleaned.csv', index=False)